In [183]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from gurobipy import *

In [184]:
import pandas as pd

In [185]:
import numpy as np

In [186]:
def PercentageOptimalityGap(optimal_value, alg_value):
    return (optimal_value - alg_value) / optimal_value

In [187]:
def int_to_string(number):
    if(number < 10):
        return '0' + str(number)
    else:
        return str(number)

In [188]:
'''
IP formulation
Input: m,n,k,b,p
Output: optimal value
'''
def IP_answer(M,N,K,B,P):
    p3_b = Model("p3_b")
    
    # a_jm
    a = {}

    for j in range(N):
        for m in range(M):
            a[j,m] = p3_b.addVar(lb = 0, vtype = GRB.BINARY, name = "a_" + str(j+1) + "_" + str(m+1))

    # w 
    w = p3_b.addVar(lb = 0, vtype= GRB.CONTINUOUS, name = "w")
    
    # obj
    p3_b.setObjective(w, GRB.MAXIMIZE)
    
    #1
    p3_b.addConstrs(((quicksum(a[j,m] for m in range(M)) <= 1) for j in range(N)), "1 job asigned to 1 machine" )

    #2
    p3_b.addConstrs(((quicksum(a[j,m]*P[j] for j in range(N)) <= K) for m in range(M)), "not exceed capacity")

    #3
    p3_b.addConstrs(((quicksum(B[j]*a[j,m] for j in range(N)) >= w) for m in range(M)), "w is the min")

    #Optimize
    p3_b.optimize()
    
    # Return Optimal Value
    return p3_b.objVal

In [189]:
'''
HBF formulation
Input: m,n,k,b,p
Output: optimal value
'''
def HBF_answer(m,n,k,b,p):
    # Init accumulated processing time and benefit of each machine
    P = [0 for i in range(m)]
    B = [0 for i in range(m)]
    
    # Sorting the job
    for i in range(n):
        for j in range(0, n-i-1):
            if b[j+1] > b[j]:
                b[j], b[j+1] = b[j+1], b[j]
                p[j], p[j+1] = p[j+1], p[j]
            
    # Main Heuristic Part
    for i in range(n):
#         temp_accumulated_benefit = B
        temp_accumulated_benefit = {}
        for j in range(m):
            temp_accumulated_benefit[j] = B[j]
        for j in range(m):
            min_value = min(temp_accumulated_benefit)
            min_index =  min(temp_accumulated_benefit, key=temp_accumulated_benefit.get)
            if(k - P[min_index] - p[i] >= 0):
                P[min_index] += p[i]
                B[min_index] += b[i]
                break
            else:
                temp_accumulated_benefit.pop(min_index, None)
                
    min_index, min_benefit = min_info(B)
    return min_benefit

In [190]:
'''
HRF formulation
Input: m,n,k,b,p
Output: optimal value
'''
def HRF_answer(m,n,k,b,p):
    # Init accumulated processing time and benefit of each machine
    P = [0 for i in range(m)]
    B = [0 for i in range(m)]
    
    # Sorting the job
    for i in range(n):
        for j in range(0, n-i-1):
            if (b[j+1]/p[j+1]) > (b[j]/p[j]):
                b[j], b[j+1] = b[j+1], b[j]
                p[j], p[j+1] = p[j+1], p[j]
            
    # Main Heuristic Part
    for i in range(n):
#         temp_accumulated_benefit = B
        temp_accumulated_benefit = {}
        for j in range(m):
            temp_accumulated_benefit[j] = B[j]
        for j in range(m):
            min_value = min(temp_accumulated_benefit)
            min_index =  min(temp_accumulated_benefit, key=temp_accumulated_benefit.get)
            if(k - P[min_index] - p[i] >= 0):
                P[min_index] += p[i]
                B[min_index] += b[i]
                break
            else:
                temp_accumulated_benefit.pop(min_index, None)
                
    min_index, min_benefit = min_info(B)
    return min_benefit

# 4-a

In [191]:
# Create Dataframe
data = {
    'IP': [0 for i in range(20)],
    'HBF': [0 for i in range(20)],
    'HRF': [0 for i in range(20)]
}
df_answer = pd.DataFrame(data)
df_answer

,IP,HBF,HRF
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
5,0,0,0
6,0,0,0
7,0,0,0
8,0,0,0
9,0,0,0


In [192]:
for i in range(20):
    filename = 'data/in' + int_to_string(i+1) + '.txt'
    with open(filename,'r') as f:
        m,n,k = f.readline().split()
        m = int(m)
        n = int(n)
        k = int(k)
        b = [int(i) for i in f.readline().split()]
        p = [int(i) for i in f.readline().split()]
    
        df_answer['IP'][i] = IP_answer(m,n,k,b,p)
        df_answer['HBF'][i] = HBF_answer(m,n,k,b,p)
        df_answer['HRF'][i] = HRF_answer(m,n,k,b,p)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 51 rows, 136 columns and 408 nonzeros
Model fingerprint: 0xac92817b
Variable types: 1 continuous, 135 integer (135 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 51 rows, 136 columns, 408 nonzeros
Variable types: 0 continuous, 136 integer (135 binary)
Found heuristic solution: objective 1.0000000

Root relaxation: objective 1.467000e+02, 84 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  146.70000    0    9    1.00000  146.70000      -     -    0s
H    0     0                     1

Model fingerprint: 0x2b52f095
Variable types: 1 continuous, 135 integer (135 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 51 rows, 136 columns, 408 nonzeros
Variable types: 0 continuous, 136 integer (135 binary)
Found heuristic solution: objective 1.0000000

Root relaxation: objective 1.443684e+02, 82 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  144.36842    0    9    1.00000  144.36842      -     -    0s
H    0     0                     109.0000000  144.36842  32.4%     -    0s
H    0     0                     143.0000000  144.36842  0.96%     -    0s
     0     0  144.00000    0    9  143.00000  144.00000  0.70%     -    0s


     0     0   83.80000    0   18   79.00000   83.80000  6.08%     -    0s
     0     0   83.80000    0   16   79.00000   83.80000  6.08%     -    0s
     0     0   83.80000    0   10   79.00000   83.80000  6.08%     -    0s
     0     0   83.80000    0   12   79.00000   83.80000  6.08%     -    0s
     0     0   83.80000    0   17   79.00000   83.80000  6.08%     -    0s
     0     0   83.80000    0   11   79.00000   83.80000  6.08%     -    0s
     0     0   83.80000    0   13   79.00000   83.80000  6.08%     -    0s
     0     0   83.80000    0   10   79.00000   83.80000  6.08%     -    0s
     0     0   83.80000    0    8   79.00000   83.80000  6.08%     -    0s
H    0     0                      80.0000000   83.80000  4.75%     -    0s
H    0     0                      83.0000000   83.80000  0.96%     -    0s
     0     0   83.80000    0    8   83.00000   83.80000  0.96%     -    0s

Cutting planes:
  MIR: 3

Explored 1 nodes (965 simplex iterations) in 0.19 seconds (0.02 work unit

H    0     0                     131.0000000  138.40000  5.65%     -    0s
H    0     0                     135.0000000  138.40000  2.52%     -    0s
     0     0  138.40000    0   15  135.00000  138.40000  2.52%     -    0s
H    0     0                     137.0000000  138.40000  1.02%     -    0s
     0     0  138.33333    0   12  137.00000  138.33333  0.97%     -    0s
     0     0  138.33333    0    6  137.00000  138.33333  0.97%     -    0s
     0     0  138.33333    0    8  137.00000  138.33333  0.97%     -    0s
     0     0  138.33333    0   12  137.00000  138.33333  0.97%     -    0s
H    0     0                     138.0000000  138.33333  0.24%     -    0s
     0     0  138.33333    0   12  138.00000  138.33333  0.24%     -    0s

Cutting planes:
  Gomory: 1
  Lift-and-project: 1
  MIR: 1

Explored 1 nodes (280 simplex iterations) in 0.11 seconds (0.01 work units)
Thread count was 4 (of 4 available processors)

Solution count 7: 138 137 135 ... -0

Optimal solution found (tol

     0     0   59.20000    0   18   56.00000   59.20000  5.71%     -    0s
     0     0   59.20000    0   17   56.00000   59.20000  5.71%     -    0s
     0     0   59.20000    0   19   56.00000   59.20000  5.71%     -    0s
     0     0   59.20000    0   14   56.00000   59.20000  5.71%     -    0s
     0     0   59.20000    0   17   56.00000   59.20000  5.71%     -    0s
     0     0   59.20000    0   13   56.00000   59.20000  5.71%     -    0s
H    0     0                      57.0000000   59.20000  3.86%     -    0s
     0     0   59.20000    0   12   57.00000   59.20000  3.86%     -    0s
     0     0   59.20000    0   13   57.00000   59.20000  3.86%     -    0s
     0     0   59.20000    0   12   57.00000   59.20000  3.86%     -    0s
     0     0   59.20000    0   18   57.00000   59.20000  3.86%     -    0s
     0     0   59.20000    0   13   57.00000   59.20000  3.86%     -    0s
     0     0   59.20000    0   12   57.00000   59.20000  3.86%     -    0s
     0     2   59.20000  

     0     0   62.00000    0   17   61.00000   62.00000  1.64%     -    0s
     0     0   62.00000    0   19   61.00000   62.00000  1.64%     -    0s
     0     0   62.00000    0   13   61.00000   62.00000  1.64%     -    0s
     0     0   62.00000    0   14   61.00000   62.00000  1.64%     -    0s
     0     0   62.00000    0   15   61.00000   62.00000  1.64%     -    0s
     0     0   62.00000    0   15   61.00000   62.00000  1.64%     -    0s
     0     0   62.00000    0   16   61.00000   62.00000  1.64%     -    0s
     0     0   62.00000    0   13   61.00000   62.00000  1.64%     -    0s
     0     2   62.00000    0    9   61.00000   62.00000  1.64%     -    0s
H  260   214                      62.0000000   62.00000  0.00%  14.8    0s

Cutting planes:
  Gomory: 3
  Cover: 28
  MIR: 25
  StrongCG: 15
  Flow cover: 22
  Inf proof: 1
  Zero half: 1
  RLT: 2

Explored 458 nodes (6820 simplex iterations) in 0.79 seconds (0.14 work units)
Thread count was 4 (of 4 available processors)



     0     0   56.00000    0   20   55.00000   56.00000  1.82%     -    0s
     0     0   56.00000    0   13   55.00000   56.00000  1.82%     -    0s
     0     0   56.00000    0   13   55.00000   56.00000  1.82%     -    0s
     0     0   56.00000    0   14   55.00000   56.00000  1.82%     -    0s
     0     0   56.00000    0   18   55.00000   56.00000  1.82%     -    0s
     0     0   56.00000    0   14   55.00000   56.00000  1.82%     -    0s
     0     0   56.00000    0   17   55.00000   56.00000  1.82%     -    0s
     0     0   56.00000    0   18   55.00000   56.00000  1.82%     -    0s
     0     0   56.00000    0   18   55.00000   56.00000  1.82%     -    0s
     0     2   56.00000    0   10   55.00000   56.00000  1.82%     -    0s
* 1338   118              18      56.0000000   56.00000  0.00%  12.7    1s

Cutting planes:
  Gomory: 1
  Cover: 242
  MIR: 8
  StrongCG: 3
  Inf proof: 2
  RLT: 2

Explored 1359 nodes (19093 simplex iterations) in 1.40 seconds (0.48 work units)
Thre

In [193]:
df_answer

,IP,HBF,HRF
0,146,144,145
1,95,89,92
2,54,50,51
3,144,136,140
4,59,49,55
5,115,108,112
6,83,76,80
7,76,70,71
8,85,80,80
9,138,137,136


In [194]:
for index, row in df_answer.iterrows():
    if(row['IP'] < row['HBF'] or row['IP'] < row['HRF']):
        print("error")
print("pass")

pass


## Calculate Percentage Optimality Gap

In [195]:
gap_HBF_and_IP = []
gap_HRF_and_IP = []
for i in range(20):
    gap_HBF_and_IP.append(PercentageOptimalityGap(df_answer['IP'][i], df_answer['HBF'][i]))
    gap_HRF_and_IP.append(PercentageOptimalityGap(df_answer['IP'][i], df_answer['HRF'][i]))

In [196]:
df_answer['HBF Gap'] = gap_HBF_and_IP
df_answer['HRF Gap'] = gap_HRF_and_IP

In [197]:
df_answer

,IP,HBF,HRF,HBF Gap,HRF Gap
0,146,144,145,0.013699,0.006849
1,95,89,92,0.063158,0.031579
2,54,50,51,0.074074,0.055556
3,144,136,140,0.055556,0.027778
4,59,49,55,0.169492,0.067797
5,115,108,112,0.060870,0.026087
6,83,76,80,0.084337,0.036145
7,76,70,71,0.078947,0.065789
8,85,80,80,0.058824,0.058824
9,138,137,136,0.007246,0.014493


In [198]:
print("Average percentage optimality gap of HBF  = ", np.mean(df_answer['HBF Gap']))
print("Average percentage optimality gap of HRF  = ", np.mean(df_answer['HRF Gap']))

Average percentage optimality gap of HBF  =  0.07095815961094817
Average percentage optimality gap of HRF  =  0.050576676721239146


# 4-b